In [1]:
import pandas as pd
import pandas_datareader.data as pdr
from pandas import Series, DataFrame
import numpy as np

import datetime
from datetime import date, timedelta
import yfinance as yf
yf.pdr_override()

import plotly.graph_objects as go
from plotly.subplots import make_subplots

def fullPrint(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        print(df)   


In [2]:
df = pdr.get_data_yahoo("GME", start="2016-01-01",end="2021-11-26")
sp = "^GSPC"
spdf = pdr.get_data_yahoo(sp, start="2016-01-01",end="2021-11-26")
sortLogic = {
    'Close': 'last',
    'Volume': 'sum'
}
dfSorted = df.resample("W-FRI").agg(sortLogic)
spDfSorted = spdf.resample("W-FRI").agg(sortLogic)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
weights = np.arange(1,31)
sumWeights = np.sum(weights)
dfSorted['30WMA'] = dfSorted['Close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)

In [4]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=dfSorted.index, y=dfSorted['Close'],name='Price'),secondary_y=True)
fig.add_trace(go.Scatter(x=dfSorted.index, y=dfSorted['30WMA'],name="30WMA"),secondary_y=True)
fig.add_trace(go.Bar(x=dfSorted.index, y=dfSorted['Volume'],name="Volume"),secondary_y=False)
fig.show()

In [5]:
deltaX = 10.4
dfSorted['30WMASlope'] = dfSorted['30WMA'].diff()*500/(dfSorted['Close'].rolling(window=104).max()-dfSorted['Close'].rolling(window=104).min())/deltaX
fullPrint(dfSorted)

                 Close     Volume       30WMA  30WMASlope
Date                                                     
2016-01-08   28.370001   16865400         NaN         NaN
2016-01-15   25.639999   22044800         NaN         NaN
2016-01-22   25.670000    8902700         NaN         NaN
2016-01-29   26.209999    9933800         NaN         NaN
2016-02-05   26.830000   10397300         NaN         NaN
2016-02-12   26.980000   11663200         NaN         NaN
2016-02-19   28.190001    8002900         NaN         NaN
2016-02-26   30.540001    7772500         NaN         NaN
2016-03-04   31.230000   14673000         NaN         NaN
2016-03-11   30.750000   11421900         NaN         NaN
2016-03-18   30.830000   13913600         NaN         NaN
2016-03-25   30.270000   11991000         NaN         NaN
2016-04-01   31.340000   18800900         NaN         NaN
2016-04-08   29.490000   11340800         NaN         NaN
2016-04-15   31.790001   11328700         NaN         NaN
2016-04-22   3

In [6]:
dfSorted['Percent'] = dfSorted['Close'].pct_change()
dfSorted['VolumePerc'] = dfSorted['Volume'].pct_change() 
spDfSorted['Percent'] = spDfSorted['Close'].pct_change()
dfSorted['RS'] = dfSorted['Percent'] - spDfSorted['Percent']
dfSorted

,Close,Volume,30WMA,30WMASlope,Percent,VolumePerc,RS
Date,,,,,,,
2016-01-08,28.370001,16865400,NaN,NaN,NaN,NaN,NaN
2016-01-15,25.639999,22044800,NaN,NaN,-0.096228,0.307102,-0.074533
2016-01-22,25.670000,8902700,NaN,NaN,0.001170,-0.596154,-0.012960
2016-01-29,26.209999,9933800,NaN,NaN,0.021036,0.115819,0.003552
2016-02-05,26.830000,10397300,NaN,NaN,0.023655,0.046659,0.054677
...,...,...,...,...,...,...,...
2021-10-29,183.509995,8712300,184.569225,-0.005744,0.080742,-0.217919,0.067435
2021-11-05,213.250000,24809200,186.466516,0.283103,0.162062,1.847606,0.142053
2021-11-12,202.100006,7318700,187.526409,0.158151,-0.052286,-0.705001,-0.049161


In [10]:
def checkIfStage2(price,volumePerc, RS, slope, WMA):
    if volumePerc < 0.2:
        return "Volume"
    if RS < 0:
        return "RS"
    if slope < 0.8:
        return "Slope"
    if price < WMA*1.1:
        return "Price"
    return "Clear"

In [11]:
dfSorted['IsStage2'] = dfSorted.apply(lambda x: checkIfStage2(x['Close'],x['VolumePerc'],x['RS'],x['30WMASlope'],x['30WMA']),axis=1)

In [9]:
fullPrint(dfSorted)

                 Close     Volume       30WMA  30WMASlope   Percent  \
Date                                                                  
2016-01-08   28.370001   16865400         NaN         NaN       NaN   
2016-01-15   25.639999   22044800         NaN         NaN -0.096228   
2016-01-22   25.670000    8902700         NaN         NaN  0.001170   
2016-01-29   26.209999    9933800         NaN         NaN  0.021036   
2016-02-05   26.830000   10397300         NaN         NaN  0.023655   
2016-02-12   26.980000   11663200         NaN         NaN  0.005591   
2016-02-19   28.190001    8002900         NaN         NaN  0.044848   
2016-02-26   30.540001    7772500         NaN         NaN  0.083363   
2016-03-04   31.230000   14673000         NaN         NaN  0.022593   
2016-03-11   30.750000   11421900         NaN         NaN -0.015370   
2016-03-18   30.830000   13913600         NaN         NaN  0.002602   
2016-03-25   30.270000   11991000         NaN         NaN -0.018164   
2016-0